In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
import itertools
import sys
sys.path.append('..')
from security_game.target import Target
from security_game.green_security_game import GreenSecurityGame
from security_game.infra_security_game import InfraSecurityGame

from solvers.mip import mip
from solvers.nash import nash
from solvers.double_oracle import double_oracle
from solvers.double_oracle_sf import double_oracle_sf
from solvers.no_regret import regret_matching
from solvers.simple_sse_lp import solve_sse_lp
from solvers.nfg_sse_lp import solve_general_sum_normal_form

from utils.random_utils import generate_random_utility_matrix_like, generate_random_target_utility_matrix_like, generate_random_schedule_mapping_like, generate_random_target_utility_matrix_like_v2

import time
from matplotlib import pyplot as plt
from collections import defaultdict, Counter
import copy
from scipy import stats

In [2]:
seeds = [1,2,3,4,5,6,7,8,9,10]

# GSG

In [40]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)
sangha_river = [(2.2837, 16.16283352464626),(2.053, 16.066212728001727)]

## Simple SSE LP - Random target values only

In [41]:
schedule_form_kwargs = {
    "schedule_form": True,
    "simple": True,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": True,
    "attacker_animal_value":  2350, 
    "defender_animal_value": 22966, 
    "defender_step_cost": 0, 
}

# general_sum_kwargs = {
#     "general_sum": True,
#     "attacker_animal_value":  1, 
#     "defender_animal_value": 1, 
#     "defender_step_cost": 0, 
# }



supports=[]
dus = []
runtimes = []

gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7, escape_line_points=sangha_river)
gsg.generate(num_attackers=1, 
             num_defenders=2, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post),(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=8, 
             generate_utility_matrix=False, 
             defense_time_threshold=1, 
             force_return=True, 
             **schedule_form_kwargs,
             **general_sum_kwargs)
for seed in seeds:
    print(f"starting seed: {seed}")
    np.random.seed(seed)
    sdict_copy = copy.deepcopy(gsg.schedule_form_dict)
    sdict_copy["target_utilities"] = generate_random_target_utility_matrix_like_v2(gsg.schedule_form_dict["target_utilities"], general_sum=True, respect_sign_roles=True)
    print(sdict_copy["target_utilities"])
    start = time.time()
    _, coverage, du = solve_sse_lp(sdict_copy)
    end=time.time()

    support = sum([1 for t,p in coverage.items() if p!=0])
    dus.append(du)
    runtimes.append(end-start)
    supports.append(support)

starting seed: 1
[[-0.42163788 -0.72150105 -0.207799   -0.87863018 -0.15034536 -0.67185381
  -0.41975612 -0.56054636 -0.39930519 -0.54075686]
 [-0.41947451 -0.68654374 -0.00432075 -0.30526756 -0.03147923 -0.09615699
  -0.1896835  -0.34831386 -0.14400321 -0.20147496]
 [ 0.10213996  0.42354294  0.3163125   0.53512919  0.69317334  0.31839516
   0.0888933   0.04309734  0.0224182   0.75119542]
 [ 0.80158281  0.96839509  0.95806668  0.69361697  0.87690917  0.89505004
   0.68781977  0.83532138  0.17332282  0.87865514]]
starting seed: 2
[[-0.62272767 -0.53112292 -0.55155698 -0.51562443 -0.42280623 -0.78623821
  -0.8545896  -0.62087264 -0.84720698 -0.26991163]
 [-0.43836759 -0.03002402 -0.13822064 -0.43769791 -0.18787081 -0.333152
  -0.20799456 -0.49636451 -0.30260092 -0.08351727]
 [ 0.47002642  0.06921871  0.43052813  0.10033168  0.13083188  0.3894719
   0.22926806  0.11070263  0.16582269  0.35256147]
 [ 0.50732745  0.20510138  0.64191948  0.48524449  0.50731812  0.59844174
   0.79450559  0.58

In [42]:
df = pd.DataFrame()
df["seed"] = seeds
df["num_timesteps"] = [8 for i in range(len(seeds))]
df["num_defenders"] = [2 for i in range(len(seeds))]
df["num_clusters"] = [10 for i in range(len(seeds))]
df["dims"] = [7 for i in range(len(seeds))]
df["defense_time_threshold"] = [1 for i in range(len(seeds))]
df["defender_utility"] = dus
df["support"] = supports
df["runtime"] = runtimes
df

,seed,num_timesteps,num_defenders,num_clusters,dims,defense_time_threshold,defender_utility,support,runtime
0,1,8,2,10,7,1,-0.097629,7,0.012651
1,2,8,2,10,7,1,-0.205678,5,0.013500
2,3,8,2,10,7,1,-0.343018,6,0.014165
3,4,8,2,10,7,1,-0.204440,1,0.012596
4,5,8,2,10,7,1,-0.358472,5,0.013375
5,6,8,2,10,7,1,-0.342275,5,0.013132
6,7,8,2,10,7,1,-0.387901,6,0.013485
7,8,8,2,10,7,1,-0.286917,6,0.014183
8,9,8,2,10,7,1,-0.428047,4,0.013301
9,10,8,2,10,7,1,-0.321545,5,0.013707


In [45]:
np.mean([round(x,4) for x in df.support])

5.0

In [46]:
stats.sem([round(x,4) for x in df.support])

0.5163977794943222

In [47]:
np.mean([round(x,4) for x in df.runtime])

0.013420000000000001

In [48]:
stats.sem([round(x,4) for x in df.runtime])

0.00017048949136725907

In [49]:
np.mean([round(x,4) for x in df.defender_utility])

-0.29758

In [50]:
stats.sem([round(x,4) for x in df.defender_utility])

0.031753189866006636

In [10]:
df.to_csv("GSG_SIMPLE_SSE_RANDOM_TARGET_VALUES.csv")

## Simple SSE LP - Entirely Random schedules

## General SSE LP - Random Matrix

In [16]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)

In [10]:
schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5
}

general_sum_kwargs = {
    "general_sum": True,
    "attacker_animal_value":  2350, 
    "defender_animal_value": 22966, 
    "defender_step_cost": 1.17, 
}


supports=[]
dus = []
runtimes = []

gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7, escape_line_points=sangha_river)
gsg.generate(num_attackers=1, 
             num_defenders=2, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post),(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=8, 
             generate_utility_matrix=True, 
             defense_time_threshold=1, 
             force_return=True, 
             **schedule_form_kwargs,
             **general_sum_kwargs)
for seed in seeds:
    print(f"starting seed: {seed}")
    np.random.seed(seed)
    sdict_copy = copy.deepcopy(gsg.schedule_form_dict)
    sdict_copy["defender_utility_matrix"] = generate_random_utility_matrix_like(gsg.schedule_form_dict["defender_utility_matrix"])
    sdict_copy["attacker_utility_matrix"] = generate_random_utility_matrix_like(gsg.schedule_form_dict["attacker_utility_matrix"])

    flat_indices = np.argpartition(sdict_copy["defender_utility_matrix"].ravel(), -5)[-5:]
    sorted_indices = flat_indices[np.argsort(sdict_copy["defender_utility_matrix"].ravel()[flat_indices])[::-1]]
    
    for idx in sorted_indices:
        row, col = np.unravel_index(idx, sdict_copy["defender_utility_matrix"].shape)
        print(f"Value: {sdict_copy["defender_utility_matrix"][row, col]}, Index: ({row}, {col})")
        
    # print(sdict_copy["defender_utility_matrix"])
    # print(sdict_copy["attacker_utility_matrix"])
    start = time.time()
    _, coverage, du = solve_general_sum_normal_form(sdict_copy["defender_utility_matrix"],sdict_copy["attacker_utility_matrix"])
    end=time.time()

    support = sum([1 for t,p in coverage.items() if p!=0])
    dus.append(du)
    runtimes.append(end-start)
    supports.append(support)

starting seed: 1
Value: -0.03473849135230922, Index: (1962, 4)
Value: -0.034761957825537926, Index: (264, 5)
Value: -0.034774759270366884, Index: (2413, 5)
Value: -0.03480208289629427, Index: (1873, 1)
Value: -0.03487628098427531, Index: (2423, 2)
starting seed: 2
Value: -0.03469714209329178, Index: (2295, 6)
Value: -0.03471994157677516, Index: (900, 0)
Value: -0.03481261092595267, Index: (1800, 4)
Value: -0.034867032164570766, Index: (873, 7)
Value: -0.034875096882631684, Index: (2334, 0)
starting seed: 3
Value: -0.03467006332057565, Index: (1552, 8)
Value: -0.03469586513618794, Index: (2400, 0)
Value: -0.03472037096567182, Index: (2164, 9)
Value: -0.03476227863362247, Index: (1516, 6)
Value: -0.03477490233847935, Index: (1632, 6)
starting seed: 4
Value: -0.03474445250649427, Index: (1032, 4)
Value: -0.03474855746927863, Index: (2387, 1)
Value: -0.03485181188842279, Index: (532, 4)
Value: -0.03487952903171432, Index: (2136, 3)
Value: -0.03489453444568691, Index: (1331, 6)
starting see

In [9]:
np.max(gsg.schedule_form_dict["defender_utility_matrix"])

-0.03464204837266125

In [19]:
df = pd.DataFrame()
df["seed"] = seeds
df["num_timesteps"] = [8 for i in range(len(seeds))]
df["num_defenders"] = [2 for i in range(len(seeds))]
df["num_clusters"] = [10 for i in range(len(seeds))]
df["dims"] = [7 for i in range(len(seeds))]
df["defense_time_threshold"] = [1 for i in range(len(seeds))]
df["defender_utility"] = dus
df["support"] = supports
df["runtime"] = runtimes
df

,seed,num_timesteps,num_defenders,num_clusters,dims,defense_time_threshold,defender_utility,support,runtime
0,1,8,2,10,7,1,-0.034781,2,0.519425
1,2,8,2,10,7,1,-0.035010,3,0.436632
2,3,8,2,10,7,1,-0.034768,2,0.601936
3,4,8,2,10,7,1,-0.034802,2,0.567851
4,5,8,2,10,7,1,-0.034800,2,0.451001
5,6,8,2,10,7,1,-0.034754,2,0.502337
6,7,8,2,10,7,1,-0.034720,1,0.499972
7,8,8,2,10,7,1,-0.034861,2,0.481408
8,9,8,2,10,7,1,-0.034830,2,0.619688
9,10,8,2,10,7,1,-0.034765,2,0.478741


In [22]:
[round(x,4) for x in df.runtime]

[0.5194, 0.4366, 0.6019, 0.5679, 0.451, 0.5023, 0.5, 0.4814, 0.6197, 0.4787]

In [17]:
df.to_csv("GSG_NFG_SSE_RANDOM_MATRIX.csv")

In [62]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)

schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": True,
    "attacker_animal_value":  2350, 
    "defender_animal_value": 22966, 
    "defender_step_cost": 1.17, 
}


supports=[]
dus = []
runtimes = []

gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7, escape_line_points=sangha_river)
gsg.generate(num_attackers=1, 
             num_defenders=2, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post),(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=8, 
             generate_utility_matrix=True, 
             random_target_values=False,
             defense_time_threshold=1, 
             force_return=True, 
             **schedule_form_kwargs,
             **general_sum_kwargs)

for seed in seeds:
    print(f"starting seed: {seed}")
    np.random.seed(seed)
    defender_actions,defender_costs = generate_defender_actions_with_costs(gsg.schedule_form_dict["schedules"])
    attacker_actions = [0,1,2,3,4,5,6,7,8,9]
    random_target_utilities = generate_random_target_utility_matrix_like_v2(gsg.schedule_form_dict["target_utilities"], general_sum=True, respect_sign_roles=True)
    print(random_target_utilities)
    attacker_matrix, defender_matrix = generate_schedule_game_matrix(attacker_actions, defender_actions, defender_costs, random_target_utilities, general_sum=True)
    start = time.time()
    _, coverage, du = solve_general_sum_normal_form(defender_matrix,attacker_matrix)
    end=time.time()

    support = sum([1 for t,p in coverage.items() if p!=0])
    dus.append(du)
    runtimes.append(end-start)
    supports.append(support)

starting seed: 1
[[-0.42163788 -0.72150105 -0.207799   -0.87863018 -0.15034536 -0.67185381
  -0.41975612 -0.56054636 -0.39930519 -0.54075686]
 [-0.41947451 -0.68654374 -0.00432075 -0.30526756 -0.03147923 -0.09615699
  -0.1896835  -0.34831386 -0.14400321 -0.20147496]
 [ 0.10213996  0.42354294  0.3163125   0.53512919  0.69317334  0.31839516
   0.0888933   0.04309734  0.0224182   0.75119542]
 [ 0.80158281  0.96839509  0.95806668  0.69361697  0.87690917  0.89505004
   0.68781977  0.83532138  0.17332282  0.87865514]]
starting seed: 2
[[-0.62272767 -0.53112292 -0.55155698 -0.51562443 -0.42280623 -0.78623821
  -0.8545896  -0.62087264 -0.84720698 -0.26991163]
 [-0.43836759 -0.03002402 -0.13822064 -0.43769791 -0.18787081 -0.333152
  -0.20799456 -0.49636451 -0.30260092 -0.08351727]
 [ 0.47002642  0.06921871  0.43052813  0.10033168  0.13083188  0.3894719
   0.22926806  0.11070263  0.16582269  0.35256147]
 [ 0.50732745  0.20510138  0.64191948  0.48524449  0.50731812  0.59844174
   0.79450559  0.58

### new target randomization

In [63]:
df = pd.DataFrame()
df["seed"] = seeds
df["num_timesteps"] = [8 for i in range(len(seeds))]
df["num_defenders"] = [2 for i in range(len(seeds))]
df["num_clusters"] = [10 for i in range(len(seeds))]
df["dims"] = [7 for i in range(len(seeds))]
df["defense_time_threshold"] = [1 for i in range(len(seeds))]
df["defender_utility"] = dus
df["support"] = supports
df["runtime"] = runtimes
df

,seed,num_timesteps,num_defenders,num_clusters,dims,defense_time_threshold,defender_utility,support,runtime
0,1,8,2,10,7,1,-0.249858,1,0.357311
1,2,8,2,10,7,1,-0.257696,1,0.376930
2,3,8,2,10,7,1,-0.252614,3,0.361606
3,4,8,2,10,7,1,-0.249463,1,0.338337
4,5,8,2,10,7,1,-0.253488,2,0.332397
5,6,8,2,10,7,1,-0.252082,2,0.413847
6,7,8,2,10,7,1,-0.256689,2,0.339143
7,8,8,2,10,7,1,-0.252432,3,0.345470
8,9,8,2,10,7,1,-0.257924,1,0.339115
9,10,8,2,10,7,1,-0.254759,1,0.348098


In [64]:
np.mean([round(x,4) for x in df.support])

1.7

In [65]:
stats.sem([round(x,4) for x in df.support])

0.2603416558635552

In [66]:
np.mean([round(x,4) for x in df.runtime])

0.35521

In [67]:
stats.sem([round(x,4) for x in df.runtime])

0.007759287624804512

In [68]:
np.mean([round(x,4) for x in df.defender_utility])

-0.25371

In [69]:
stats.sem([round(x,4) for x in df.defender_utility])

0.0009511922343389194

In [22]:
df.to_csv("GSG_NFG_SSE_RANDOM_TARGET_VALUES.csv")

In [3]:
    def generate_defender_actions_with_costs(schedule_dict):
        """
        Generates all possible joint defender actions from a dictionary mapping
        each defender to their list of (schedule, cost) tuples.

        Defenders with no schedules are assigned an empty set with 0 cost as a placeholder.

        Returns:
            defender_actions: list of lists of sets (each inner list is one full defender action)
            defender_costs: list of floats representing the total cost for each joint action
        """
        sorted_defenders = sorted(schedule_dict.keys())
        # print(schedule_dict)
        # Replace empty schedule lists with a dummy no-op schedule
        schedule_lists = [
            schedule_dict[d] if schedule_dict[d] else [({}, 0)]
            for d in sorted_defenders
        ]
        # print(schedule_lists)
        all_combinations = list(itertools.product(*schedule_lists))

        defender_actions = []
        defender_costs = []

        for combo in all_combinations:
            schedules = [item[0] for item in combo]
            # print(combo)
            total_cost = sum(item[1] for item in combo)
            defender_actions.append(schedules)
            defender_costs.append(total_cost)

        return defender_actions, defender_costs

    def generate_schedule_game_matrix(attacker_actions, defender_actions, defender_costs, target_utility_matrix, general_sum):
        """
        Builds utility matrices for all combinations of defender and attacker actions,
        then normalizes both matrices together based on the largest absolute value.

        Parameters:
            attacker_actions (list[int]): List of target nodes (attacker action).
            defender_actions (list[list[set]]): Each outer list is one defender action;
                inner list is per-defender selected schedule (a set of targets).
            defender_costs (list[float]): Cost associated with each defender action.
            target_utility_matrix (np.ndarray): 4 x num_targets array:
                [0]: defender utility if uncovered
                [1]: defender utility if covered
                [2]: attacker utility if covered
                [3]: attacker utility if uncovered

        Returns:
            tuple:
                - attacker_util_matrix (np.ndarray): shape (len(defender_actions), len(attacker_actions))
                - defender_util_matrix (np.ndarray): same shape
        """
        num_defender_actions = len(defender_actions)
        num_attacker_actions = len(attacker_actions)

        defender_util_matrix = np.zeros((num_defender_actions, num_attacker_actions))
        attacker_util_matrix = np.zeros((num_defender_actions, num_attacker_actions))

        for i, d_action in enumerate(defender_actions):
            target_coverage_count = {}
            for schedule in d_action:
                for t in schedule:
                    target_coverage_count[t] = target_coverage_count.get(t, 0) + 1

            for j, atk_target in enumerate(attacker_actions):
                num_covers = target_coverage_count.get(atk_target, 0)

                if num_covers == 0:
                    defender_util = target_utility_matrix[0][j]  # uncovered
                    attacker_util = target_utility_matrix[3][j]  # uncovered
                else:
                    defender_util = target_utility_matrix[1][j]  # covered
                    attacker_util = target_utility_matrix[2][j]  # covered

                defender_util_matrix[i, j] = defender_util - defender_costs[i]
                # defender_util_matrix[i, j] = defender_util
                attacker_util_matrix[i, j] = attacker_util

        # if general_sum:
            # Normalize attacker and defender matrices together
        max_abs_value = max(
            np.abs(defender_util_matrix).max(),
            np.abs(attacker_util_matrix).max()
        )

        if max_abs_value > 0:
            defender_util_matrix = defender_util_matrix / max_abs_value
            attacker_util_matrix = attacker_util_matrix / max_abs_value

        return attacker_util_matrix, defender_util_matrix

In [79]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)

schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": True,
    "attacker_animal_value":  2350, 
    "defender_animal_value": 22966, 
    "defender_step_cost": 1.17, 
}


supports=[]
dus = []
runtimes = []

gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7, escape_line_points=sangha_river)
gsg.generate(num_attackers=1, 
             num_defenders=2, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post),(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=8, 
             generate_utility_matrix=False, 
             random_target_values=False,
             defense_time_threshold=1, 
             force_return=True, 
             **schedule_form_kwargs,
             **general_sum_kwargs)

for seed in seeds:
    print(f"starting seed: {seed}")
    np.random.seed(seed)

    random_schedules = generate_random_schedule_mapping_like(gsg.schedule_form_dict["schedules"],48)
    defender_actions, defender_costs = generate_defender_actions_with_costs(random_schedules)
    attacker_actions = [0,1,2,3,4,5,6,7,8,9]
    random_target_utilities = generate_random_target_utility_matrix_like_v2(gsg.schedule_form_dict["target_utilities"], general_sum=True, respect_sign_roles=True)
    attacker_matrix, defender_matrix = generate_schedule_game_matrix(attacker_actions, defender_actions, defender_costs, random_target_utilities, general_sum=True)
    start = time.time()
    _, coverage, du = solve_general_sum_normal_form(defender_matrix,attacker_matrix)
    end=time.time()

    support = sum([1 for t,p in coverage.items() if p!=0])
    dus.append(du)
    runtimes.append(end-start)
    supports.append(support)


starting seed: 1
starting seed: 2
starting seed: 3
starting seed: 4
starting seed: 5
starting seed: 6
starting seed: 7
starting seed: 8
starting seed: 9
starting seed: 10


### new randomization logic

In [80]:
df = pd.DataFrame()
df["seed"] = seeds
df["num_timesteps"] = [8 for i in range(len(seeds))]
df["num_defenders"] = [2 for i in range(len(seeds))]
df["num_clusters"] = [10 for i in range(len(seeds))]
df["dims"] = [7 for i in range(len(seeds))]
df["defense_time_threshold"] = [1 for i in range(len(seeds))]
df["defender_utility"] = dus
df["support"] = supports
df["runtime"] = runtimes
df

,seed,num_timesteps,num_defenders,num_clusters,dims,defense_time_threshold,defender_utility,support,runtime
0,1,8,2,10,7,1,-0.257217,1,0.348460
1,2,8,2,10,7,1,-0.285653,1,0.510406
2,3,8,2,10,7,1,-0.275671,2,0.427433
3,4,8,2,10,7,1,-0.259953,1,0.473227
4,5,8,2,10,7,1,-0.258675,1,0.401918
5,6,8,2,10,7,1,-0.289263,1,0.382147
6,7,8,2,10,7,1,-0.278246,2,0.457276
7,8,8,2,10,7,1,-0.265214,2,0.410036
8,9,8,2,10,7,1,-0.261432,1,0.436831
9,10,8,2,10,7,1,-0.268206,2,0.640203


In [81]:
np.mean([round(x,4) for x in df.support])

1.4

In [82]:
stats.sem([round(x,4) for x in df.support])

0.16329931618554522

In [83]:
np.mean([round(x,4) for x in df.runtime])

0.44878

In [84]:
stats.sem([round(x,4) for x in df.runtime])

0.025800972419228265

In [85]:
np.mean([round(x,4) for x in df.defender_utility])

-0.26996000000000003

In [86]:
stats.sem([round(x,4) for x in df.defender_utility])

0.003667369629584671

### old randomizaton logic

In [30]:
df = pd.DataFrame()
df["seed"] = seeds
df["num_timesteps"] = [8 for i in range(len(seeds))]
df["num_defenders"] = [2 for i in range(len(seeds))]
df["num_clusters"] = [10 for i in range(len(seeds))]
df["dims"] = [7 for i in range(len(seeds))]
df["defense_time_threshold"] = [1 for i in range(len(seeds))]
df["defender_utility"] = dus
df["support"] = supports
df["runtime"] = runtimes
df

,seed,num_timesteps,num_defenders,num_clusters,dims,defense_time_threshold,defender_utility,support,runtime
0,1,8,2,10,7,1,-0.364794,2,0.433395
1,2,8,2,10,7,1,-0.041760,1,0.437731
2,3,8,2,10,7,1,-0.335759,3,0.362280
3,4,8,2,10,7,1,-0.137561,2,0.341587
4,5,8,2,10,7,1,-0.262375,2,0.380102
5,6,8,2,10,7,1,-0.069592,3,0.350575
6,7,8,2,10,7,1,-0.077671,1,0.364267
7,8,8,2,10,7,1,-0.295773,3,0.501500
8,9,8,2,10,7,1,-0.237512,2,0.391941
9,10,8,2,10,7,1,-0.133573,4,0.385948


In [31]:
df.to_csv("GSG_NFG_SSE_RANDOM_SCHEDULES_AND_TARGET_VALUES.csv")

# ISG

In [5]:
gdf = gpd.read_file("chinatown_infra.geojson")

# Step 1: Handle relevant columns
infra_columns = [
    "id", "name", "power", "man_made", "amenity",
    "generator:method", "generator:source", "geometry"
]
available_columns = [col for col in infra_columns if col in gdf.columns]
gdf = gdf[available_columns].copy()

# Step 2: Extract generator type if present
gdf["generator_type"] = gdf.get("generator:method")
if "generator_type" not in gdf.columns or gdf["generator_type"].isnull().all():
    gdf["generator_type"] = gdf.get("generator:source")

# Step 3: Construct unified 'type' column
gdf["type"] = gdf.get("power")
if "amenity" in gdf.columns:
    gdf["type"] = gdf["type"].combine_first(gdf["amenity"])
if "man_made" in gdf.columns:
    gdf["type"] = gdf["type"].combine_first(gdf["man_made"])

# Step 4: Refine generator classification (solar vs. other)
gdf.loc[(gdf["type"] == "generator") & (gdf["generator_type"] == "photovoltaic"), "type"] = "solar_generator"
gdf.loc[(gdf["type"] == "generator") & (gdf["generator_type"] == "solar"), "type"] = "solar_generator"

# Step 5: Drop raw columns now that 'type' is finalized
df_simple = gdf[["id", "name", "type", "geometry"]].copy()

# Step 6: Separate nodes and ways
df_nodes = df_simple[df_simple["id"].str.contains("node")].copy()
df_nodes["x"] = df_nodes.geometry.x
df_nodes["y"] = df_nodes.geometry.y
df_nodes = df_nodes.drop(columns=["geometry"])

df_ways = df_simple[df_simple["id"].str.contains("way")].copy()
df_ways = df_ways.set_geometry("geometry").to_crs("EPSG:32618")
df_ways["centroid"] = df_ways.geometry.centroid
df_ways = df_ways.set_geometry("centroid").to_crs("EPSG:4326")
df_ways["x"] = df_ways.geometry.x
df_ways["y"] = df_ways.geometry.y
df_ways = df_ways.drop(columns=["geometry", "centroid"])

# Step 7: Combine nodes and ways
df_combined = pd.concat([df_nodes, df_ways], ignore_index=True)
df_combined = pd.concat([df_nodes, df_ways], ignore_index=True)
ny_blocks_gdf =  gpd.read_file("tl_2020_36_tabblock20.shp")
INFRA_WEIGHTS = {
    # Power Infrastructure
    "plant": 1.5,
    "generator": 1.35,
    "solar_generator": 0.95,
    "substation": 1.45,
    "transformer": 1.25,
    "tower": 1.1,
    "pole": 0.85,
    "line": 1.0,
    "minor_line": 0.9,
    "cable": 0.95,
    "switchgear": 1.2,
    "busbar": 0.8,
    "bay": 0.85,
    "converter": 1.05,
    "insulator": 0.75,
    "portal": 0.75,
    "connection": 0.7,
    "compensator": 1.0,
    "rectifier": 0.95,
    "inverter": 0.95,
    "storage": 0.9,

    # Healthcare
    "hospital": 1.5,
    "clinic": 1.35,

    # Education
    "school": 1.25,
    "university": 1.4,

    # Water & Sanitation
    "water_works": 1.45,
    "wastewater_plant": 1.4,

    # Government & Emergency Services
    "fire_station": 1.3,
    "police": 1.4,
    "courthouse": 1.2,

    # Critical Infrastructure
    "bunker_silo": 1.0,

    # Communications
    "communications_tower": 1.25,
}


## ISG Simple SSE LP

In [54]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

schedule_form_kwargs = {
    "schedule_form": True,
    "simple": True,
    "attacker_penalty_factor": 3,
    "defender_penalty_factor": 3
}

general_sum_kwargs = {
    "general_sum": True,
    "attacker_feature_value":  1, 
    "defender_feature_value": 100, 
    "defender_step_cost": 0,
    "alpha":.5
}

# Bounding box for Hoboken, NJ
# bbox_hoboken_small = (40.752635, 40.745600, -74.030386,-74.043903)
bbox_hoboken_low = (40.745411, 40.735486, -74.025857,-74.041479)
bbox_hoboken_east = (40.748337, 40.734641,-74.022961,-74.031286)
bbox_downtown = (40.718721, 40.714078, -73.996074, -74.002651)
bbox_downtown_large = (40.7215, 40.710, -73.9935, -74.010)
# 40.7060, -74.0140, 40.7205, -73.9935
college_police = (40.743293077312465, -74.02670221027175)
police_station = (40.73768931976651, -74.02990519431108)
traffic_police = (40.7366602084371, -74.03449866349136)
downtown_station = (40.71232433042349, -74.00187755238431)
fifth_ave_station = (40.71637413934789, -73.9973285259067)
fifth_precinct = (40.71625547686622, -73.99736909131171)
booking_station = (40.716191530904815, -74.00102237385177)
police_plaza = (40.71236124409745, -74.00173715463521)
troop_nyc = (40.71657885026091, -74.00641139014367)
first_precinct = (40.720411300417446, -74.0070247584372)

num_timesteps_li = []
num_defenders_li = []
supports=[]
dts = []

dus = []
runtimes = []
i=0

isg = InfraSecurityGame(df_combined, ny_blocks_gdf, INFRA_WEIGHTS, bbox=bbox_downtown_large)
isg.generate(num_attackers=1, 
             num_defenders=3, 
             home_base_assignments=[(fifth_precinct,booking_station, troop_nyc, first_precinct, police_plaza),(fifth_precinct,booking_station, troop_nyc, first_precinct, police_plaza),(fifth_precinct,booking_station, troop_nyc, first_precinct, police_plaza)], 
             num_timesteps=8, 
             generate_utility_matrix=False, 
             generate_actions=False, 
             force_return=True, 
             defense_time_threshold=1, 
             **schedule_form_kwargs,
             **general_sum_kwargs, 
            )
for seed in seeds:
    print(f"starting seed: {seed}")
    np.random.seed(seed)
    sdict_copy = copy.deepcopy(isg.schedule_form_dict)
    sdict_copy["target_utilities"] = generate_random_target_utility_matrix_like_v2(isg.schedule_form_dict["target_utilities"], general_sum=True, respect_sign_roles=True)
    
    start = time.time()
    _, coverage, du = solve_sse_lp(sdict_copy)
    end=time.time()

    support = sum([1 for t,p in coverage.items() if p!=0])
    dus.append(du)
    runtimes.append(end-start)
    supports.append(support)

starting seed: 1
starting seed: 2
starting seed: 3
starting seed: 4
starting seed: 5
starting seed: 6
starting seed: 7
starting seed: 8
starting seed: 9
starting seed: 10


In [55]:
df = pd.DataFrame()
df["seed"] = seeds
df["num_timesteps"] = [8 for i in range(len(seeds))]
df["num_defenders"] = [3 for i in range(len(seeds))]
df["defense_time_threshold"] = [1 for i in range(len(seeds))]
df["defender_utility"] = dus
df["support"] = supports
df["runtime"] = runtimes
df

,seed,num_timesteps,num_defenders,defense_time_threshold,defender_utility,support,runtime
0,1,8,3,1,-0.317088,2,0.027678
1,2,8,3,1,-0.112543,12,0.040828
2,3,8,3,1,-0.297338,13,0.041553
3,4,8,3,1,-0.074156,9,0.051275
4,5,8,3,1,-0.082098,4,0.059753
5,6,8,3,1,-0.123084,10,0.030066
6,7,8,3,1,-0.307583,6,0.028065
7,8,8,3,1,-0.549742,4,0.027598
8,9,8,3,1,-0.182052,10,0.030777
9,10,8,3,1,-0.089624,5,0.028890


In [16]:
[round(x,4) for x in df.support]

[2, 2, 1, 1, 1, 4, 2, 1, 5, 1]

In [56]:
np.mean([round(x,4) for x in df.support])

7.5

In [57]:
stats.sem([round(x,4) for x in df.support])

1.194896555262328

In [58]:
np.mean([round(x,4) for x in df.runtime])

0.036669999999999994

In [59]:
stats.sem([round(x,4) for x in df.runtime])

0.0035971609175385337

In [60]:
np.mean([round(x,4) for x in df.defender_utility])

-0.21353

In [61]:
stats.sem([round(x,4) for x in df.defender_utility])

0.04842192122949641

In [9]:
df.to_csv("ISG_SIMPLE_SSE_RANDOM_TARGET_VALUES.csv")

In [33]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 3,
    "defender_penalty_factor": 3
}

general_sum_kwargs = {
    "general_sum": True,
    "attacker_feature_value":  1, 
    "defender_feature_value": 100, 
    "defender_step_cost": 2,
    "alpha":.5
}

# Bounding box for Hoboken, NJ
# bbox_hoboken_small = (40.752635, 40.745600, -74.030386,-74.043903)
bbox_hoboken_low = (40.745411, 40.735486, -74.025857,-74.041479)
bbox_hoboken_east = (40.748337, 40.734641,-74.022961,-74.031286)
bbox_downtown = (40.718721, 40.714078, -73.996074, -74.002651)
bbox_downtown_large = (40.7215, 40.710, -73.9935, -74.010)
# 40.7060, -74.0140, 40.7205, -73.9935
college_police = (40.743293077312465, -74.02670221027175)
police_station = (40.73768931976651, -74.02990519431108)
traffic_police = (40.7366602084371, -74.03449866349136)
downtown_station = (40.71232433042349, -74.00187755238431)
fifth_ave_station = (40.71637413934789, -73.9973285259067)
fifth_precinct = (40.71625547686622, -73.99736909131171)
booking_station = (40.716191530904815, -74.00102237385177)
police_plaza = (40.71236124409745, -74.00173715463521)
troop_nyc = (40.71657885026091, -74.00641139014367)
first_precinct = (40.720411300417446, -74.0070247584372)

supports=[]
dts = []

dus = []
runtimes = []

isg = InfraSecurityGame(df_combined, ny_blocks_gdf, INFRA_WEIGHTS, bbox=bbox_downtown_large)
isg.generate(num_attackers=1, 
             num_defenders=3, 
             home_base_assignments=[(fifth_precinct,booking_station, troop_nyc, first_precinct, police_plaza),(fifth_precinct,booking_station, troop_nyc, first_precinct, police_plaza)], 
             num_timesteps=8, 
             generate_utility_matrix=True, 
             generate_actions=False, 
             force_return=True, 
             defense_time_threshold=2, 
             **schedule_form_kwargs,
             **general_sum_kwargs, 
            )
for seed in seeds:
    print(f"starting seed: {seed}")
    np.random.seed(seed)
    sdict_copy = copy.deepcopy(isg.schedule_form_dict)
    sdict_copy["defender_utility_matrix"] = generate_random_utility_matrix_like(isg.schedule_form_dict["defender_utility_matrix"])
    sdict_copy["attacker_utility_matrix"] = generate_random_utility_matrix_like(isg.schedule_form_dict["attacker_utility_matrix"])
    print(sdict_copy["defender_utility_matrix"])
    print(sdict_copy["attacker_utility_matrix"])
    start = time.time()
    _, coverage, du = solve_general_sum_normal_form(sdict_copy["defender_utility_matrix"],sdict_copy["attacker_utility_matrix"])
    end=time.time()

    support = sum([1 for t,p in coverage.items() if p!=0])
    dus.append(du)
    runtimes.append(end-start)
    supports.append(support)

starting seed: 1
[[-0.59169758 -0.2947369  -0.99988802 ... -0.21599834 -0.05198398
  -0.69312926]
 [-0.32215328 -0.14193542 -0.12409883 ... -0.22722385 -0.89893236
  -0.56147156]
 [-0.11040248 -0.71252508 -0.71824181 ... -0.35008476 -0.4958768
  -0.07515594]
 ...
 [-0.62150302 -0.07923594 -0.22096152 ... -0.34961872 -0.33088097
  -0.92943706]
 [-0.81947367 -0.43827116 -0.51085942 ... -0.81773173 -0.65734987
  -0.53180536]
 [-0.09006962 -0.92660242 -0.53653057 ... -0.14301337 -0.96011702
  -0.42188867]]
[[0.00270674 0.00285909 0.00902488 ... 0.00370563 0.00952263 0.00672378]
 [0.00855286 0.0079319  0.00828676 ... 0.00071188 0.00067955 0.0012529 ]
 [0.00306844 0.00713365 0.00960149 ... 0.00160479 0.00218471 0.00463471]
 ...
 [0.00350636 0.00487196 0.00447379 ... 0.00628288 0.00880423 0.00918377]
 [0.00524711 0.00264302 0.00951103 ... 0.00617418 0.00084619 0.00446367]
 [0.00929346 0.00667294 0.00887731 ... 0.00376071 0.00107242 0.00782623]]
starting seed: 2
[[-0.57312139 -0.97461586 -0.46

In [34]:
df = pd.DataFrame()
df["seed"] = seeds
df["num_timesteps"] = [8 for i in range(len(seeds))]
df["num_defenders"] = [3 for i in range(len(seeds))]
df["defense_time_threshold"] = [1 for i in range(len(seeds))]
df["defender_utility"] = dus
df["support"] = supports
df["runtime"] = runtimes
df

,seed,num_timesteps,num_defenders,defense_time_threshold,defender_utility,support,runtime
0,1,8,3,1,-0.020930,2,2.130541
1,2,8,3,1,-0.021068,2,2.216249
2,3,8,3,1,-0.021014,1,2.185663
3,4,8,3,1,-0.021333,3,2.403166
4,5,8,3,1,-0.020914,1,1.997962
5,6,8,3,1,-0.020962,1,2.882317
6,7,8,3,1,-0.021171,1,2.685996
7,8,8,3,1,-0.021207,4,2.930520
8,9,8,3,1,-0.020962,1,2.315993
9,10,8,3,1,-0.020994,2,1.839676


In [35]:
df.to_csv("ISG_NFG_SSE_RANDOM_MATRIX.csv")

In [13]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 3,
    "defender_penalty_factor": 3
}

general_sum_kwargs = {
    "general_sum": True,
    "attacker_feature_value":  1, 
    "defender_feature_value": 100, 
    "defender_step_cost": 2,
    "alpha":.5
}

# Bounding box for Hoboken, NJ
# bbox_hoboken_small = (40.752635, 40.745600, -74.030386,-74.043903)
bbox_hoboken_low = (40.745411, 40.735486, -74.025857,-74.041479)
bbox_hoboken_east = (40.748337, 40.734641,-74.022961,-74.031286)
bbox_downtown = (40.718721, 40.714078, -73.996074, -74.002651)
bbox_downtown_large = (40.7215, 40.710, -73.9935, -74.010)
# 40.7060, -74.0140, 40.7205, -73.9935
college_police = (40.743293077312465, -74.02670221027175)
police_station = (40.73768931976651, -74.02990519431108)
traffic_police = (40.7366602084371, -74.03449866349136)
downtown_station = (40.71232433042349, -74.00187755238431)
fifth_ave_station = (40.71637413934789, -73.9973285259067)
fifth_precinct = (40.71625547686622, -73.99736909131171)
booking_station = (40.716191530904815, -74.00102237385177)
police_plaza = (40.71236124409745, -74.00173715463521)
troop_nyc = (40.71657885026091, -74.00641139014367)
first_precinct = (40.720411300417446, -74.0070247584372)

supports=[]
dts = []

dus = []
runtimes = []

isg = InfraSecurityGame(df_combined, ny_blocks_gdf, INFRA_WEIGHTS, bbox=bbox_downtown_large)
isg.generate(num_attackers=1, 
             num_defenders=3, 
             home_base_assignments=[(fifth_precinct,booking_station, troop_nyc, first_precinct, police_plaza),(fifth_precinct,booking_station, troop_nyc, first_precinct, police_plaza),(fifth_precinct,booking_station, troop_nyc, first_precinct, police_plaza)], 
             num_timesteps=8, 
             generate_utility_matrix=False, 
             random_target_values=False,
             randomize_target_utility_matrix=False,
             generate_actions=False, 
             force_return=True, 
             defense_time_threshold=1, 
             **schedule_form_kwargs,
             **general_sum_kwargs, 
            )

# for seed in seeds:
#     print(f"starting seed: {seed}")
#     np.random.seed(seed)
    
#     print(isg.schedule_form_dict["target_utilities"])
#     start = time.time()
#     _, coverage, du = solve_general_sum_normal_form(isg.schedule_form_dict["defender_utility_matrix"],isg.schedule_form_dict["attacker_utility_matrix"])
#     end=time.time()

#     support = sum([1 for t,p in coverage.items() if p!=0])
#     dus.append(du)
#     runtimes.append(end-start)
#     supports.append(support)

for seed in seeds:
    print(f"starting seed: {seed}")
    np.random.seed(seed)
    defender_actions,defender_costs = generate_defender_actions_with_costs(isg.schedule_form_dict["schedules"])
    attacker_actions = [0,1,2,3,4,5,6,7,8,9]
    random_target_utilities = generate_random_target_utility_matrix_like_v2(isg.schedule_form_dict["target_utilities"], general_sum=True, respect_sign_roles=True)
    print(random_target_utilities)
    attacker_matrix, defender_matrix = generate_schedule_game_matrix(attacker_actions, defender_actions, defender_costs, random_target_utilities, general_sum=True)
    start = time.time()
    _, coverage, du = solve_general_sum_normal_form(defender_matrix,attacker_matrix)
    end=time.time()

    support = sum([1 for t,p in coverage.items() if p!=0])
    dus.append(du)
    runtimes.append(end-start)
    supports.append(support)

starting seed: 1
[[-6.92381775e-01 -8.76412920e-01 -8.94626928e-01 -3.02466718e-01
  -1.46919949e-01 -1.69990042e-01 -8.78165934e-01 -3.45686560e-01
  -4.21218932e-01 -9.57897627e-01 -5.33255046e-01 -6.91936359e-01
  -3.15647241e-01 -8.78140872e-01 -8.34657469e-01 -6.70530871e-01
  -7.50192356e-01 -9.88863231e-01 -7.48214076e-01 -2.80582346e-01
  -8.00782881e-01 -9.68267678e-01 -4.47999683e-01]
 [-4.17134098e-01 -7.20378268e-01 -3.06629178e-04 -8.52201357e-02
  -3.92395503e-02 -9.25131166e-02 -1.86416674e-01 -9.85202004e-02
  -3.96883462e-01 -5.38905409e-01 -4.19306190e-01 -6.85280025e-01
  -2.04605215e-01 -6.86561206e-01 -2.75746036e-02 -1.84770373e-02
  -4.17416841e-01 -5.58774682e-01 -1.40552222e-01 -1.98255675e-01
  -7.89319845e-01 -1.03398435e-01 -3.13556190e-01]
 [ 5.86634536e-01  2.93749970e-01  1.37640547e-01  1.30195847e-01
   1.95555104e-02  3.97792649e-01  1.65514680e-01  2.65687877e-01
   3.47891269e-01  5.35445611e-02  5.74199493e-01  1.46892639e-01
   5.89384504e-01  6.99

KeyboardInterrupt: 

In [7]:
len(defender_actions)

636056

In [8]:
df = pd.DataFrame()
df["seed"] = seeds
df["num_timesteps"] = [8 for i in range(len(seeds))]
df["num_defenders"] = [3 for i in range(len(seeds))]
df["defense_time_threshold"] = [1 for i in range(len(seeds))]
df["defender_utility"] = dus
df["support"] = supports
df["runtime"] = runtimes
df

,seed,num_timesteps,num_defenders,defense_time_threshold,defender_utility,support,runtime
0,1,8,3,1,-0.007061,1,111.553856
1,2,8,3,1,-0.008952,1,126.751138
2,3,8,3,1,-0.010432,1,114.891145
3,4,8,3,1,-0.014247,1,112.728036
4,5,8,3,1,-0.005182,1,117.559772
5,6,8,3,1,-0.012147,1,111.024647
6,7,8,3,1,-0.018562,1,112.103105
7,8,8,3,1,-0.017831,1,113.998473
8,9,8,3,1,-0.016409,1,126.918250
9,10,8,3,1,-0.018579,1,114.693679


In [9]:
np.mean([round(x,4) for x in df.support])

1.0

In [10]:
stats.sem([round(x,4) for x in df.support])

0.0

In [11]:
np.mean([round(x,4) for x in df.runtime])

116.22221000000002

In [12]:
stats.sem([round(x,4) for x in df.runtime])

1.868127209649635

In [13]:
np.mean([round(x,4) for x in df.defender_utility])

-0.012939999999999998

In [14]:
stats.sem([round(x,4) for x in df.defender_utility])

0.0015561490931141526

In [9]:
df.to_csv("ISG_NFG_SSE_RANDOM_TARGET_VALUES.csv")

# Random target vals and schedules

In [15]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 3,
    "defender_penalty_factor": 3
}

general_sum_kwargs = {
    "general_sum": True,
    "attacker_feature_value":  1, 
    "defender_feature_value": 100, 
    "defender_step_cost": 2,
    "alpha":.5
}

# Bounding box for Hoboken, NJ
# bbox_hoboken_small = (40.752635, 40.745600, -74.030386,-74.043903)
bbox_hoboken_low = (40.745411, 40.735486, -74.025857,-74.041479)
bbox_hoboken_east = (40.748337, 40.734641,-74.022961,-74.031286)
bbox_downtown = (40.718721, 40.714078, -73.996074, -74.002651)
bbox_downtown_large = (40.7215, 40.710, -73.9935, -74.010)
# 40.7060, -74.0140, 40.7205, -73.9935
college_police = (40.743293077312465, -74.02670221027175)
police_station = (40.73768931976651, -74.02990519431108)
traffic_police = (40.7366602084371, -74.03449866349136)
downtown_station = (40.71232433042349, -74.00187755238431)
fifth_ave_station = (40.71637413934789, -73.9973285259067)
fifth_precinct = (40.71625547686622, -73.99736909131171)
booking_station = (40.716191530904815, -74.00102237385177)
police_plaza = (40.71236124409745, -74.00173715463521)
troop_nyc = (40.71657885026091, -74.00641139014367)
first_precinct = (40.720411300417446, -74.0070247584372)

supports=[]
dts = []

dus = []
runtimes = []

isg = InfraSecurityGame(df_combined, ny_blocks_gdf, INFRA_WEIGHTS, bbox=bbox_downtown_large)
isg.generate(num_attackers=1, 
             num_defenders=3, 
             home_base_assignments=[(fifth_precinct,booking_station, troop_nyc, first_precinct, police_plaza),(fifth_precinct,booking_station, troop_nyc, first_precinct, police_plaza),(fifth_precinct,booking_station, troop_nyc, first_precinct, police_plaza)], 
             num_timesteps=8, 
             generate_utility_matrix=False, 
             random_target_values=False,
             randomize_target_utility_matrix=False,
             generate_actions=False, 
             force_return=True, 
             defense_time_threshold=1, 
             **schedule_form_kwargs,
             **general_sum_kwargs, 
            )

# for seed in seeds:
#     print(f"starting seed: {seed}")
#     np.random.seed(seed)
    
#     print(isg.schedule_form_dict["target_utilities"])
#     start = time.time()
#     _, coverage, du = solve_general_sum_normal_form(isg.schedule_form_dict["defender_utility_matrix"],isg.schedule_form_dict["attacker_utility_matrix"])
#     end=time.time()

#     support = sum([1 for t,p in coverage.items() if p!=0])
#     dus.append(du)
#     runtimes.append(end-start)
#     supports.append(support)

for seed in seeds:
    print(f"starting seed: {seed}")
    np.random.seed(seed)
    random_schedules = generate_random_schedule_mapping_like(isg.schedule_form_dict["schedules"],86)
    defender_actions, defender_costs = generate_defender_actions_with_costs(random_schedules)
    attacker_actions = [0,1,2,3,4,5,6,7,8,9]
    random_target_utilities = generate_random_target_utility_matrix_like_v2(isg.schedule_form_dict["target_utilities"], general_sum=True, respect_sign_roles=True)
    attacker_matrix, defender_matrix = generate_schedule_game_matrix(attacker_actions, defender_actions, defender_costs, random_target_utilities, general_sum=True)
    start = time.time()
    _, coverage, du = solve_general_sum_normal_form(defender_matrix,attacker_matrix)
    end=time.time()

    support = sum([1 for t,p in coverage.items() if p!=0])
    dus.append(du)
    runtimes.append(end-start)
    supports.append(support)

starting seed: 1
starting seed: 2
starting seed: 3
starting seed: 4
starting seed: 5
starting seed: 6
starting seed: 7
starting seed: 8
starting seed: 9
starting seed: 10


In [16]:
df = pd.DataFrame()
df["seed"] = seeds
df["num_timesteps"] = [8 for i in range(len(seeds))]
df["num_defenders"] = [3 for i in range(len(seeds))]
df["defense_time_threshold"] = [1 for i in range(len(seeds))]
df["defender_utility"] = dus
df["support"] = supports
df["runtime"] = runtimes
df

,seed,num_timesteps,num_defenders,defense_time_threshold,defender_utility,support,runtime
0,1,8,3,1,-0.028684,1,1.514182
1,2,8,3,1,-0.031275,1,1.626506
2,3,8,3,1,-0.038951,1,1.443529
3,4,8,3,1,-0.021896,1,1.630756
4,5,8,3,1,-0.025304,1,1.670459
5,6,8,3,1,-0.055399,1,1.541387
6,7,8,3,1,-0.022354,1,1.698927
7,8,8,3,1,-0.030878,1,1.776669
8,9,8,3,1,-0.026281,1,1.302374
9,10,8,3,1,-0.033919,1,1.534827


In [17]:
np.mean([round(x,4) for x in df.support])

1.0

In [18]:
stats.sem([round(x,4) for x in df.support])

0.0

In [19]:
np.mean([round(x,4) for x in df.runtime])

1.57397

In [20]:
stats.sem([round(x,4) for x in df.runtime])

0.04332747152660641

In [21]:
np.mean([round(x,4) for x in df.defender_utility])

-0.031509999999999996

In [22]:
stats.sem([round(x,4) for x in df.defender_utility])

0.0031303336705356005

In [23]:
df.to_csv("ISG_NFG_SSE_RANDOM_SCHEDULES_AND_TARGET_VALUES.csv")

In [27]:
df_combined

,id,name,type,x,y,population
0,node/357579732,Public School 23,school,-73.998750,40.716214,419.0
1,node/357610557,Public School 124,school,-73.997500,40.714444,92.0
2,node/357621626,Public School 130,school,-73.998489,40.718536,376.0
3,node/368045750,New York City Criminal Courts Building,courthouse,-74.000806,40.715986,173.0
4,node/368045773,New York City Supreme Court,courthouse,-74.007640,40.717324,0.0
...,...,...,...,...,...,...
59,way/891074473,Pace University,university,-74.004840,40.711193,340.0
60,way/1082129088,None,solar_generator,-73.994953,40.712414,1443.0
61,way/1365373854,Stuyvesant High School,school,-74.013826,40.718015,753.0
62,way/1157729510,Weill Medical College of Cornell University,university,-74.004956,40.710307,16.0


In [30]:
gdf = None

In [29]:
gdf

,id,@id,NY_mta:propcode,access,addr:city,addr:country,addr:flats,addr:floor,addr:housename,addr:housenumber,...,website:en,website_archive,wheelchair,wikidata,wikimedia_commons,wikipedia,@relations,wires,year_of_construction,geometry
0,relation/3105305,relation/3105305,None,None,None,None,None,None,None,None,...,None,None,None,Q1048851,None,en:Yeshiva University,None,None,None,"MULTIPOLYGON (((-73.92917 40.85230, -73.92894 ..."
1,relation/3262857,relation/3262857,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-73.98941 40.68964, -73.98944 40.689..."
2,relation/3262986,relation/3262986,None,None,Brooklyn,None,None,None,None,301,...,None,None,None,Q5440041,None,en:Federal Building and Post Office (Brooklyn),None,None,None,"POLYGON ((-73.98910 40.69548, -73.98910 40.695..."
3,relation/3336423,relation/3336423,None,None,None,None,None,None,None,5404,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-73.92524 40.64756, -73.92532 40.648..."
4,relation/3351361,relation/3351361,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-73.99197 40.63115, -73.99203 40.631..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23602,node/12736543648,node/12736543648,None,None,Newark,None,None,None,None,189,...,None,None,None,None,None,None,None,None,None,POINT (-74.17123 40.73550)
23603,node/12754254335,node/12754254335,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,POINT (-74.12037 40.74476)
23604,node/12754254336,node/12754254336,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,POINT (-74.12048 40.74485)
23605,node/12762992538,node/12762992538,None,None,Bronx,None,None,None,None,1118,...,None,None,yes,None,None,None,None,None,None,POINT (-73.91899 40.83230)
